In [1]:
import copy
import numpy as np
import pandas as pd

from src.common.functions import get_feature_importance
%cd /home/heza7322/PycharmProjects/missing-value-handling-in-carts
from src.binary_tree import BinaryTree
from src.trinary_tree import TrinaryTree
from src.weighted_tree import WeightedTree
from src.common.functions import get_indices, calculate_loss, fit_response

/home/heza7322/PycharmProjects/missing-value-handling-in-carts


### Create data

In [2]:
n = 10000
seed = 11
np.random.seed(seed)
df = pd.DataFrame(index = range(n))
df['cont_normal'] = np.random.normal(3,2,n)
df['cont_linear'] = np.arange(n)
df['cont_stairs'] = np.concatenate([np.ones(int(i)) * i for i in np.linspace(0,10000,10)])[:n]
df['cat_version'] = np.random.choice(['basic','pro','extra','none'],n,p = [0.5,0.22,0.18,0.1])
df['cat_gender'] = np.random.choice(['male','female'],n,p = [0.59,1-0.59])

# Reduce unique value space
df['cont_normal'] = df['cont_normal'].round(1)
df['cont_stairs'] = df['cont_stairs'] .round(1)
df['cont_linear'] = np.floor(df['cont_linear']/10)*10

features = ['cont_normal','cont_linear','cont_stairs','cat_version','cat_gender']

### True tree structure

In [3]:
left_00 = df['cont_normal']<df['cont_normal'].quantile(0.7)
left_10 = df['cat_version'].isin(['basic','extra'])
left_11 = df['cat_gender']=='male'
left_20 = df['cont_stairs'] < df['cont_stairs'].quantile(0.4)
left_21 = df['cont_stairs'] < df['cont_stairs'].quantile(0.4)
left_22 = df['cat_version'].isin(['basic','none'])
left_23 = df['cont_linear'] < df['cont_linear'].mean()

index_30 = left_00 & left_10 & left_20
index_31 = left_00 & left_10 & (~left_20)
index_32 = left_00 & (~left_10) & left_21
index_33 = left_00 & (~left_10) & (~left_21)
index_34 = (~left_00) & left_11 & left_22
index_35 = (~left_00) & left_11 & (~left_22)
index_36 = (~left_00) & (~left_11) & left_23
index_37 = (~left_00) & (~left_11) & (~left_23)

terminal_node_indices = [index_30, index_31, index_32, index_33, index_34, index_35, index_36, index_37]
mus = np.arange(8)*10

for index,mu in zip(terminal_node_indices,mus):
    df.loc[index,'y'] = np.random.normal(mu,1)
    
# Test train split
df['test'] = False
index_train = np.random.choice(df.index,int(n*0.2))
df.loc[index_train,'test'] = True

### Hyperparameters

In [4]:
max_depth = 3
min_samples_leaf = 5
tree_types = {'majority': BinaryTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf,missing_rule='majority'),
              'mia':      BinaryTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf,missing_rule='mia'),
              'trinary': TrinaryTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf),
              'weighted': WeightedTree(max_depth=max_depth, min_samples_leaf=min_samples_leaf)}

### Example 1: No missing data

In [7]:
# Fit trees_0
trees_0 = {}
for tree_name in tree_types:
    trees_0[tree_name] = copy.deepcopy(tree_types[tree_name])
    trees_0[tree_name].fit(df.loc[~df['test'],features],df.loc[~df['test'],'y'])
    df.loc[:, f'y_hat_{tree_name}'] = trees_0[tree_name].predict(df[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df[squared_errors] = df[predictions].subtract(df['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

                    train          test
se_majority  3.589419e-29  3.633867e-29
se_mia       3.589419e-29  3.633867e-29
se_trinary   3.589419e-29  3.633867e-29
se_weighted  3.589419e-29  3.633867e-29


### Example 2: MCAR in entire dataset

In [5]:
missing_prob = 0.5
n_to_remove = int(missing_prob*n)
df_mcar_0 = df.copy()

for feature in features:
    to_remove = df_mcar_0.loc[~df_mcar_0[feature].isna()].sample(n_to_remove).index
    df_mcar_0.loc[to_remove,feature] = np.nan

# Fit trees_1
trees_1 = {}
for tree_name in tree_types:
    trees_1[tree_name] = copy.deepcopy(tree_types[tree_name])
    trees_1[tree_name].fit(df_mcar_0.loc[~df_mcar_0['test'],features],df_mcar_0.loc[~df_mcar_0['test'],'y'])
    df_mcar_0.loc[:, f'y_hat_{tree_name}'] = trees_1[tree_name].predict(df_mcar_0[features])

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_mcar_0[squared_errors] = df_mcar_0[predictions].subtract(df_mcar_0['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_mcar_0.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_mcar_0.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

                  train        test
se_majority  280.196975  279.313777
se_mia       262.118131  259.413081
se_trinary   261.448324  259.827863
se_weighted  310.589013  311.533369


### Example 3: MCAR in test set

In [8]:
df_mcar_1 = df[features+['y','test']].copy()
df_mcar_1.loc[df_mcar_1['test'],features] = df_mcar_0.loc[df_mcar_0['test'],features].copy()
    
# Predict
for tree_name in tree_types:
    df_mcar_1.loc[:, f'y_hat_{tree_name}'] = trees_0[tree_name].predict(df_mcar_1[features])

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_mcar_1[squared_errors] = df_mcar_1[predictions].subtract(df_mcar_1['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_mcar_1.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_mcar_1.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

                    train        test
se_majority  3.589419e-29  358.602238
se_mia       3.589419e-29  358.602238
se_trinary   3.589419e-29  247.764116
se_weighted  3.589419e-29  246.028204


### Example 4: MCAR for one feature

In [ ]:
feature = 'cont_normal'
df_mcar_2 = df[features+['y','test']].copy()
df_mcar_2.loc[:,feature] = df_mcar_0.loc[:,feature].copy()

# Fit trees_2
trees_2 = {}
for tree_name in tree_types:
    trees_2[tree_name] = copy.deepcopy(tree_types[tree_name])
    trees_2[tree_name].fit(df_mcar_2.loc[~df_mcar_2['test'],features],df_mcar_2.loc[~df_mcar_2['test'],'y'])
    df_mcar_2.loc[:, f'y_hat_{tree_name}'] = trees_2[tree_name].predict(df_mcar_2[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_mcar_2[squared_errors] = df_mcar_2[predictions].subtract(df_mcar_2['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_mcar_2.loc[~df_mcar_2['test'],squared_errors].mean(axis=0)
df_res['test']  = df_mcar_2.loc[df_mcar_2['test'],squared_errors].mean(axis=0)
print(df_res)

### Example 5: MAR for cont_normal

In [ ]:
def sigm(x):
    return 1/(1+np.exp(-x))

In [ ]:
missing_prob = sigm(df['cont_normal']-5)
df_mcorr_0 = df[features+['y','test']].copy()
to_remove = np.random.binomial(1,missing_prob) == 1
df_mcorr_0.loc[to_remove, 'cont_normal'] = np.nan

# Fit trees_3
trees_3 = {}
for tree_name in tree_types:
    trees_3[tree_name] = copy.deepcopy(tree_types[tree_name])
    trees_3[tree_name].fit(df_mcorr_0.loc[~df_mcorr_0['test'],features],df_mcorr_0.loc[~df_mcorr_0['test'],'y'])
    df_mcorr_0.loc[:, f'y_hat_{tree_name}'] = trees_3[tree_name].predict(df_mcorr_0[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_mcorr_0[squared_errors] = df_mcorr_0[predictions].subtract(df_mcorr_0['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_mcorr_0.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_mcorr_0.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

### Example 6: MAR for cont_normal only in test data

In [ ]:
df_mcorr_1 = df[features+['y','test']].copy()
df_mcorr_1.loc[df_mcorr_1['test'],features] = df_mcorr_0.loc[df_mcorr_0['test'],features].copy()

# Predict
for tree_name in tree_types:
    df_mcorr_1.loc[:, f'y_hat_{tree_name}'] = trees_0[tree_name].predict(df_mcorr_1[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_mcorr_1[squared_errors] = df_mcorr_1[predictions].subtract(df_mcorr_1['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_mcorr_1.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_mcorr_1.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

### Example 7: IM for cont_normal

In [ ]:
missing_prob = sigm(df['y']-5)
df_im_0 = df[features+['y','test']].copy()
to_remove = np.random.binomial(1,missing_prob) == 1
df_im_0.loc[to_remove, 'cont_normal'] = np.nan

# Fit trees_4
trees_4 = {}
for tree_name in tree_types:
    trees_4[tree_name] = copy.deepcopy(tree_types[tree_name])
    trees_4[tree_name].fit(df_im_0.loc[~df_im_0['test'],features],df_im_0.loc[~df_im_0['test'],'y'])
    df_im_0.loc[:, f'y_hat_{tree_name}'] = trees_4[tree_name].predict(df_im_0[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_im_0[squared_errors] = df_im_0[predictions].subtract(df_im_0['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_im_0.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_im_0.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)

### Example 8: IM for cont_normal only in test data

In [ ]:
df_im_1 = df[features+['y','test']].copy()
df_im_1.loc[df_im_1['test'],features] = df_im_0.loc[df_im_0['test'],features].copy()

# Predict
for tree_name in tree_types:
    df_im_1.loc[:, f'y_hat_{tree_name}'] = trees_0[tree_name].predict(df_im_1[features])
    

# Calculate squared errors
predictions = [f'y_hat_{tree_name}' for tree_name in tree_types]
squared_errors = [f'se_{tree_name}' for tree_name in tree_types]
df_im_1[squared_errors] = df_im_1[predictions].subtract(df_im_1['y'],axis=0).pow(2)

# Check results
df_res = pd.DataFrame(columns = ['train','test'])
df_res['train'] = df_im_1.loc[~df['test'],squared_errors].mean(axis=0)
df_res['test']  = df_im_1.loc[df['test'],squared_errors].mean(axis=0)
print(df_res)


In [ ]:
pd.Series(data = get_feature_importance(trees_4['trinary']))
